In [1]:
import sys
import time
import datetime
from train import *
from params import TrainingParams
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

from params import TransferParams
from pipeline import *
import numpy as np

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:279: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [8]:
params = TrainingParams()
# aws
# params.train_path='/home/ubuntu/work/data/unlabeled2017/*.jpg'
# mbp
# params.train_path='/Users/paul/Work/ai/images/val2017/*.jpg'
# azure
# params.train_path = '/home/paul/src/images/train2017/*.jpg'
# tf
params.train_path = 'data/train/val/*.tfr'
params.style_path='data/mosaic.jpg'
params.batch_size = 4
params.num_epoch = 1
params.learn_rate = 0.0001
params.total_train_sample = 400
params.style_weight = 5.0
params.content_weight = 1.0
params.tv_weight=0.0

In [3]:
def time_remaining(params, batch, epoch):
    time_remaining = 0
    if params.last_time is None:
        params.last_time = time.perf_counter()
    else:
        total_epoch = params.num_epoch
        if params.restore_epoch is not None:
            total_epoch += params.restore_epoch + 1
        cur_time = time.perf_counter()
        elapsed = cur_time - params.last_time
        params.last_time = cur_time
        batches_per_epoch = (params.total_train_sample/params.batch_size)
        batch_remaining = batches_per_epoch*(total_epoch-epoch) - batch
        time_remaining = int(batch_remaining*elapsed)
    return time_remaining

In [4]:
def keep_last(data, entry):
#     if len(data) > 20:
#         data = data[1:]
    data.append(entry)
    return data

In [5]:
def report_loss(params, batch, epoch, total_cost, content_cost, style_cost):
    params.cost_history = keep_last(params.cost_history, total_cost)
    params.style_history = keep_last(params.style_history, style_cost)
    params.content_history =keep_last(params.content_history, content_cost)
    t = time_remaining(params, batch, epoch)
    plt.title("Batch %i of epoch %i. Total Loss: %s. ETA %s " % (batch, epoch, str(total_cost), str(datetime.timedelta(seconds=t))))
    plt.plot(params.cost_history, 'r', params.style_history, 'b', params.content_history, 'g')
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [6]:
def report_loss_simple(params, batch, epoch, total_cost, content_cost, style_cost):
    t = time_remaining(params, batch, epoch)
    sys.stdout.write("\r")
    sys.stdout.write("Batch %i of epoch %i. Total Loss: %s. ETA %s " % (batch, epoch, str(total_cost), str(datetime.timedelta(seconds=t))))
    sys.stdout.flush()

In [ ]:
params.cost_history = []
params.style_history = []
params.content_history = []
params.last_time = None
params.restore_epoch = None
train(params, report_fn=report_loss_simple, start_new=False)

Evaluating Target Style...
Defining Input Pipeline...
Building Model...
Defining Losses...
Starting...
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from summaries/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into summaries/model.ckpt.
Batch 1100 of epoch 0. Total Loss: 2.09759e+11. ETA -1 day, 23:56:12 INFO:tensorflow:global_step/sec: 4.28687
Batch 1200 of epoch 0. Total Loss: 1.77978e+11. ETA -1 day, 23:55:49 INFO:tensorflow:global_step/sec: 4.37719
Batch 1300 of epoch 0. Total Loss: 1.75192e+11. ETA -1 day, 23:55:27 INFO:tensorflow:global_step/sec: 4.37707
Batch 1400 of epoch 0. Total Loss: 1.50769e+11. ETA -1 day, 23:55:03 INFO:tensorflow:global_step/sec: 4.3774
Batch 1500 of epoch 0. Total Loss: 1.56709e+11. ETA -1 day, 23:54:40 INFO:tensorflow:global_step/sec: 4.37734
Batch 1600 of epoch 0. Total Loss: 1.63083e+11. ETA -1 day, 23:54:17 INFO:tensorflow:global_step/sec: 4.37666
Batch 1700 of epoch 0. Total Loss: 1.48124e+11. ETA -1 

In [ ]:
def transfer(path, out_path=None):
    # init    
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    
    input_image = process_img(path, crop=False).eval()
    input_shape = [1] + list(input_image.shape)
    
    # make the model
    input_placeholder = tf.placeholder(dtype=tf.float32, shape=input_shape, name='input_images')
    gen = SpriteGenerator(input_placeholder, 'SpriteGenerator')
    
    # restore
    saver = tf.train.Saver()
    saver.restore(sess, 'data/model/style.ckpt')
    
    # generate and write out
    output = sess.run(gen.output, feed_dict={input_placeholder:np.stack([input_image])})
    plt.imshow(output[0]/255.)  
    
    if out_path is not None:
        output = tf.image.encode_jpeg(output[0])  
        write = tf.write_file(out_path, output)
        sess.run(write)
        print('Generate Done.')

In [ ]:
# transfer('data/test.jpg', 'data/generated.jpg')